## OJS Custom Reviews Export

Code for generating a CSV file for all submissions during a given date range from a local version of the database. Runs on Python3. Additional authors are printed within "{}".

In [ ]:
import pymysql.cursors

In [ ]:
# Connect to the database
db = pymysql.connect(host='localhost',
                            user='root',
                            password='',
                            db='ojs2',
                            cursorclass=pymysql.cursors.DictCursor)

In [ ]:
journal_id = 10

In [ ]:
# For submission: ID | Title | Author | Abstract (if possible) |  Section (if possible)

In [ ]:
# Article IDs
import csv
articles = []
with open('ITLCP-articleIDS.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        articles.append(', '.join(row))

In [ ]:
# print(articles)
article_string = ",".join( map(str, articles) )

In [ ]:
len(articles)

In [ ]:
# Get Review Information
with db.cursor() as cursor:
    #Get Review Assignments
    sql = """
        SELECT review_assignments.review_id, review_assignments.reviewer_id, 
            review_assignments.recommendation, review_assignments.submission_id,
            review_form_responses.review_form_element_id,
            review_form_responses.response_value
        FROM review_assignments
        LEFT JOIN (review_form_responses)
            on (review_assignments.review_id = review_form_responses.review_id)
        WHERE submission_id 
        IN (%s)
    """ 
    cursor.execute(sql % (article_string))
    review_responses = cursor.fetchall()

In [ ]:
# review_responses

In [ ]:
element_ids = []
for d in review_responses:
    if d['review_form_element_id'] != None:
        element_ids.append(d['review_form_element_id'])
    else:
        continue
        

In [ ]:
# print(element_ids)
element_string = ",".join( map(str, element_ids) )

In [ ]:
# Get Review Questions
with db.cursor() as cursor:
    sql = """
        SELECT review_form_element_id, setting_value
        FROM review_form_element_settings
        WHERE review_form_element_id
        IN (%s)
        AND setting_name LIKE 'question'
    """ 
    cursor.execute(sql % element_string)
    review_elements = cursor.fetchall()

In [ ]:
for each in review_elements:
    each['question'] = each.pop('setting_value')

In [ ]:
# review_elements

In [ ]:
# Get Submission Information

In [ ]:
with db.cursor() as cursor:
    sql = """
        SELECT articles.user_id, articles.section_id, articles.article_id,
            article_settings.setting_value
        FROM articles
        LEFT JOIN (article_settings)
            ON (articles.article_id = article_settings.article_id)
        WHERE articles.article_id
        IN (%s)
        AND article_settings.setting_name LIKE 'cleanTitle'
    """ 
    cursor.execute(sql % article_string)
    article_elements = cursor.fetchall()

In [ ]:
for each in article_elements:
    each['title'] = each.pop('setting_value')
    each['submission_id'] = each.pop('article_id')

In [ ]:
# article_elements

In [ ]:
# Get section information

In [ ]:
sections = []
for each in article_elements:
    sections.append(each['section_id'])
    
sections_string = ",".join( map(str, sections) )

In [ ]:
with db.cursor() as cursor:
    sql = """SELECT section_settings.setting_value, section_settings.section_id
        FROM section_settings
        WHERE section_settings.section_id 
        IN (%s)
        AND setting_name LIKE 'title'"""
    cursor.execute(sql % sections_string)
    section_names = cursor.fetchall()

In [ ]:
for each in section_names:
    each['section'] = each.pop('setting_value')

In [ ]:
# User Information

In [ ]:
users = []
for each in article_elements:
    users.append(each['user_id'])
    
users = ",".join( map(str, users) )

In [ ]:
with db.cursor() as cursor:
    sql = """
        SELECT first_name, last_name, email, user_id
        FROM users
        WHERE users.user_id
        IN (%s)
    """ 
    cursor.execute(sql % users)
    user_information = cursor.fetchall()

In [ ]:
# user_information

In [ ]:
import pandas as pd

In [ ]:
reviews = pd.DataFrame(review_responses)

In [ ]:
# reviews

In [ ]:
elements = pd.DataFrame(review_elements)

In [ ]:
# elements

In [ ]:
submissions = pd.DataFrame(article_elements)

In [ ]:
# submissions

In [ ]:
users = pd.DataFrame(user_information)

In [ ]:
# users

In [ ]:
sections = pd.DataFrame(section_names)

In [ ]:
# sections

In [ ]:
merged = pd.merge(reviews, elements, on='review_form_element_id')

In [ ]:
merged = pd.merge(merged, submissions, on='submission_id')

In [ ]:
merged = pd.merge(merged, users, on='user_id')

In [ ]:
merged = pd.merge(merged, sections, on='section_id')

In [ ]:
response_key = [{'response_value': '1', 'rating_value': 'Proposal meets the criteria.'}, 
                {'response_value': '2', 'rating_value': 'Needs some revisions prior to acceptance. (Please specify revisions.)'},
                {'response_value': '3', 'rating_value': 'Proposal does not meet the criteria.'}]

In [ ]:
response_keys = pd.DataFrame(response_key)

In [ ]:
merged = pd.merge(merged, response_keys, on='response_value', how='outer')
# merged

In [ ]:
recommendation_keys = [{'recommendation': 1, 'recommendation_value': 'Accept Submission'},
                      {'recommendation': 2, 'recommendation_value': 'Revisions Required'},
                      {'recommendation': 3, 'recommendation_value': 'Resubmit for Review'},
                      {'recommendation': 4, 'recommendation_value': 'Resubmit Elsewhere'},
                      {'recommendation': 5, 'recommendation_value': 'Decline Submission'},
                      {'recommendation': 6, 'recommendation_value': 'See Comments'}]

In [ ]:
recommendation_keys = pd.DataFrame(recommendation_keys)

In [ ]:
# recommendation_keys

In [ ]:
merged = pd.merge(merged, recommendation_keys, on='recommendation', how='outer')

In [ ]:
merged.to_csv('review-export-2016-04-26-2.csv')